<a href="https://colab.research.google.com/github/NathanDietrich/Artificial-Intelligence-and-Machine-Learning-portfolio/blob/main/ReccomendationEnginesRestart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install stuff

In [4]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=9a4d89c23a5a5d756e0060f2bcfc6f200c3bddd93a9db41f6d7d951ed8db7c87
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [27]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [22]:
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')

df = pd.merge(ratings_df, movies_df[['movieId', 'genres']], on='movieId', how ='left' )

df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [23]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()

df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = movie_encoder.fit_transform(df['movieId'])

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')), columns = mlb.classes_, index = df.index ))

df.head()

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,4.0,964982703,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,2,4.0,964981247,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0,5,4.0,964982224,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,43,5.0,964983815,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,46,5.0,964982931,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [28]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
71243,454,436,3.0,836436259,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
47082,306,1805,2.5,1186161761,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
67904,437,3225,3.0,1105669901,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
46634,305,4858,3.5,1518380959,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
43975,293,126,3.0,966596725,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66714,428,406,3.0,828124615,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
76398,479,976,2.5,1179178987,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
80192,503,2102,3.5,1063644403,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8832,61,1575,4.5,1521822456,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df.drop(columns = "(no genres listed)", inplace = True)

In [29]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [30]:
model_svd = SVD()
model_svd.fit(trainset)

predictions_svd = model_svd.test(trainset.build_anti_testset())
accuracy.rmse(predictions_svd)

RMSE: 0.4721


0.4721073405268303

In [37]:
def get_top_n_reccomendations(user_id, n=5):
  user_movies = df[df['userId'] == user_id]['movieId'].unique()
  all_movies = df['movieId'].unique()
  movies_to_predict = list(set(all_movies) - set(user_movies))
  user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
  predictions_cf = model_svd.test(user_movie_pairs)

  top_n_reccomendations = sorted(predictions_cf, key=lambda x: x.est, reverse=True)[:n]
  for pred in top_n_reccomendations:
    predicted_rating = pred.est
    print(predicted_rating)
  top_n_movie_ids = [int(pred.iid) for pred in top_n_reccomendations]
  top_n_movies = movie_encoder.inverse_transform(top_n_movie_ids)

  return top_n_movies

In [38]:
user_id = 1
reccomendations = get_top_n_reccomendations(user_id)
top_n_movies_titles = movies_df[movies_df['movieId'].isin(reccomendations)]['title'].tolist()
print(f"Top 5 reccomendations for user {user_id}:")
for i, title in enumerate(top_n_movies_titles, 1):
    print(f"{i}. {title}")

4.554863937557374
4.517695827647719
4.398807576818868
4.376089247798162
4.368689111581489
Top 5 reccomendations for user 1:
1. Pulp Fiction (1994)
2. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
3. Some Like It Hot (1959)
4. Rosemary's Baby (1968)
5. Outlaw Josey Wales, The (1976)
